# MATH20014 Mathematical Programming: Group 3 Project Submission 

Group Members: Hamad Babur, Ryan Lam, Jiawei Li, Sherry Shen 


## Table of Contents

- [Introduction](#Intro)
- Part A: [Introduction to `PyGame`](#PartA)
    - Section I: [Animating Ball Motion](#S1)
    - Section II: [Sierpenski Triangle](#S2)
    - Section III: [Fractals and Trees](#S3)
- Part B: [Applications of `Pygame`](#PartB)
    - Section I: [Julia and Mandelbrot Sets](#S4)
    - Section II: [Tower of Hanoi](#S5)
- [Conclusion](#Conc)
- [References and Bibilography](#Bib)

## Introduction <a name='Intro'></a>


Recursion is one of the most powerful tools in programming. It is not only a rather intuitive mathematical concept, but it is also easy to implement in any programming language. 

In this Python project, we will both visually and mathematically appreciate the power of recursion.  We will focus on a set of modules collectively known as `PyGame` [(Shinners, 2021)](#PyGame_Intro). They provide a concise way for us to produce interactive animations within Python.

The language of recursion naturally leads to the study of fractals, that is, self-repeating patterns and geometric objects. As we will see later, a lot of patterns in nature are also closely related to fractals.


The first part (part A) of the project consists of three short parts, aiming to understand the power of the `PyGame` module.

Firstly, in section A.I, we introduce and develop a simple pygame animation function `bouncing_ball`, which is one or more balls bouncing in a square box in two dimensions with various constraints.

In section A.II, we will introduce fractals through the Sierpinski triangle and make an animation of how the Sierpinski triangles are produced.

In section A.III, we explore a particular construction and animation of a recursively defined “tree”.

The second part (part B) of the project will focus on some interesting applications when we integrate `PyGame` with different programming techniques.

In section B.I, we shall look at Julia sequences and we shall animate the Julia set as a transition from one picture to another. In addition to this, we shall look at Mandelbrot sequences, and hence create a program that zooms in and out of the Mandelbrot set.

Finally, in section B.II, we explore a classical puzzle - the Tower of Hanoi. We will briefly look at the solution, the animation, and other time and space complexity issues.


In [1]:
# Required Imports:
import pygame, timeit
import matplotlib.pyplot as plt
import numpy as np

pygame 2.1.2 (SDL 2.0.18, Python 3.9.7)
Hello from the pygame community. https://www.pygame.org/contribute.html


<a id='PartA'></a>
## Part A: Introduction to `PyGame`


In this part, we will demonstrate animating and drawing in `PyGame`, which is very useful for our applications in [part B](#PartB).

<a id='S1'></a>
### Section I: Animating Ball Motion 


In this section, we will include some brief code to outline the main functionality of PyGame.
For this, recall that we are given a pygame animation function bouncing_ball, and we will further develop it such that:

1.	The user should be able to change the original position of the ball and should be able to change the speed of the ball.
2.	The ball should slow down under the effect of gravity.
3.	Two or more balls bounce within the same square (and of one another).


To do this, we can utilize the power of classes in Python. We can observe that intuitively, a moving ball should have a size, its current position and its current velocity, and we can make the following definition:


In [2]:
class Ball(object):
    '''
    The class Ball encapsulate all the information needed to 
    draw a ball on the PyGame module.
    '''
    def __init__(self, x, y, x_step, y_step, size=50):
        '''
        Initialization of the class Ball. 
        (x, y) - the coordinate relative to the screen, 
        both of them are float from 0 to 1.
        (x_step, y_step) - the velocity of the ball (pixel per frame).
        size - optional argument, size of the ball.
        '''
        self.x = x
        self.y = y
        self.x_step = x_step
        self.y_step = y_step
        self.size = size

Note that we set the attribute `size` as optional, as this would simplify the initialization of the object a bit.

Now we are ready to adapt the code such that gravity is implemented for *one bouncing ball*. Note that in `Pygame`, a coordinate grid is implemented. And the $y$-axis $y=0$ is at the top of the screen, that’s why when gravity is negative, the ball slows down at the top.

In [3]:
def bouncing_ball_ext(ball, speed_factor=5,gravity=9.81):
    '''
    Play a bouncing ball animation according to the inputs:
    Inputs:
    ball - an object from the class Ball
    speed_factor - the frames per second of animation
    gravity - the drag factor affecting the motion of the ball (positive is upwards)
    (N.B., gravity = 0 corresponds to no gravity, obviously)
    Output: None
    '''
    # Required variables, x0, y0 is the starting coordinate of the ball
    screen_size = (screen_width, screen_height) = (1080, 800)
    white = (255,255,255)
    x0, y0 = (screen_width - ball.size)*ball.x, (screen_height - ball.size)*ball.y
    g = -gravity
    # Used for the pause time in the animation while loop below
    frames_per_second = 10 + 10*speed_factor
    clock = pygame.time.Clock()
    # Set up the animation     
    pygame.init()
    screen = pygame.display.set_mode(screen_size)
    # Titles and Instructions
    caption = f'Bouncing Ball with Two Balls with gravity factor {-gravity}'
    caption += '                              '
    caption += '(Keystroke:  \'Space\' to start or pause, \'ArrowUp/ArrowDown\' to speed up/down)'
    pygame.display.set_caption(caption)
    # Load the image
    ball_fig = pygame.image.load("intro_ball.gif")
    ball_fig = pygame.transform.scale(ball_fig, (ball.size, ball.size))
    # Assign an pygame.Rect object to the ball
    ball_rect = pygame.Rect(x0,y0,ball.size,ball.size)
    # Initalize the Screen
    screen.fill(white)
    screen.blit(ball_fig, ball_rect)
    pygame.display.flip()
    # Variable to detect change
    keep_running = True
    move_ball = False
    # Animation loop 
    while keep_running:
        # If a keyboard event happens register it... 
        for event in pygame.event.get():
            if event.type == pygame.QUIT:
                keep_running = False
            # Start/Stop Animation
            elif event.type == pygame.KEYDOWN and event.key == pygame.K_SPACE:
                move_ball = not move_ball
            # Speed change if ArrowUp/ArrowDown is pressed
            elif event.type == pygame.KEYDOWN and event.key == pygame.K_UP:
                frames_per_second = (4/3)* frames_per_second
            elif event.type == pygame.KEYDOWN and event.key == pygame.K_DOWN:
                frames_per_second = 0.75 * frames_per_second
        if move_ball:
            # Move the ball a step under gravity
            ball.y_step += g/(frames_per_second)
            ball_rect.x += ball.x_step
            ball_rect.y += ball.y_step
            if ball_rect.left < 0 or ball_rect.right > screen_width:
                ball.x_step = - ball.x_step
            if ball_rect.top < 0 or ball_rect.bottom > screen_height:
                ball.y_step = - ball.y_step
        # Redraw and update the screen
        screen.fill(white)
        screen.blit(ball_fig, ball_rect)
        pygame.display.flip()
        clock.tick(frames_per_second)
    pygame.quit()
    return None 

We can test our function using one instance of our class `Ball`.

In [4]:
# A ball positioned at the middle, with velocity (1.2, 3).
ballA = Ball(0.5, 0.5, 1.2, 3)
bouncing_ball_ext(ballA)

The forward horizontal step size is  x_step = 1.2
The forward vertical step size is    y_step = 3


Next, we can implement collision between two balls. 

We assume every collision is *elastic*, that is,  we only have to exchange the velocity between the two balls when a collision is detected.

<a id='S2'></a>
### Section II: Sierpenski Triangle 


<a id='S3'></a>
### Section III: Fractals and Trees 

<a id='PartB'></a>
## Part B: Applications of `PyGame` 

<a id='S4'></a>
### Section I: Julia and Mandelbrot Sets 

<a id='S5'></a>
### Section II: Tower of Hanoi 

<a id='Conc'></a>
## Conclusion 

<a id='Bib'></a>
## References and Bibilography

<a id='PyGame_Intro'>(Shinners, 2021)</a> Shinners, P. (2021). *Pygame Intro — Pygame v2.1.1 Documentation.* [online] Pygame.org. Available at: http://www.pygame.org/docs/tut/PygameIntro.html [Accessed 5 May 2022].